In [ ]:
import os
import sys
import time

from scipy.optimize import Bounds, minimize

from simulationClass import FIMS_Simulation

simDir = os.getcwd()
analysisDir = os.path.join(simDir, '..', 'Analysis')
sys.path.append(analysisDir)
from runDataClass import runData

In [ ]:
def scanParam(parameters, startValues, stopValues, numSteps):
    """
    Executes a parameter scan by running the simulation multiple times with
    varying combinations of specified parameters.

    Executes an N-dimensional linear scan with every combination of parameters.
    If a single simulation fails, its run number is set to -1 and the scan continues.

    Args:
        parameters (list): List of strings for parameters to be scanned.
        startValues (list): List of numbers; Starting value of each parameter.
        stopValues (list): List of numbers; Ending value of each parameter.
        numSteps (list): List of ints; Number of steps within each parameter's range.

    Returns:
        list: List of the run numbers of the executed simulations.
              Empty if the parameters are not recognized.
    """

    if not (len(parameters) == len(startValues) == len(stopValues) == len(numSteps)):
        raise ValueError('Inputs must have the same length.')

    simulation = FIMS_Simulation()
    allParams = simulation.defaultParam()

    for inParam in parameters:
        if inParam not in allParam:
            print(f"Parameter '{inParam}' not able to be varied.")
            return []
    
    #Find range for each parameter
    allRanges = [
        np.linspace(start, stop, step) for start, stop, step in zip(startValues, stopValues, numSteps)
    ]

    print(f'Beginning scan through {parameters}. {math.prod(numSteps)} runs.')

    runNos = []
    for scanParams in itertools.product(*allRanges):

        #Reset parameters to defaults
        simulation.resetParam(varbose=False)
        
        #Apply current parameter combination
        for i, inParam in enumerate(parameters):
            simulation.param[inParam] = scanParams[i]

        runNo = simulation.runSimulation()
        if runNo == -1:
            print(f'Simulation failed with settings: {simulation}')
        runNos.append(runNo)

    print(f'Done scan through {parameters}. Final run number: {runNos[-1]}')
    simulation.resetParam()
        
    return runNos

In [ ]:
def scanValues(parameterDefinitions):
    """
    Executes a parameter scan by running the simulation multiple times with
    varying combinations of specified parameters.

    Executes an N-dimensional linear scan with every combination of parameters.
    If a single simulation fails, its run number is set to -1 and the scan continues.

    Args:
        parameterDefinitions (list): List of lists with the following form: 
                                     ['parameter', minValue, maxValue, stepValue]
                                     'parameter' - String of the parameter name.
                                      startValue - Number for the starting parameter value.
                                      stopValue - Number for the ending parameter value.
                                      stepValue - Integer number of steps through range.

    Returns:
        list: List of the run numbers of the executed simulations.
              Empty if the parameters are not recognized.
    """
    # Use a list comprehension to validate the input first
    if not all(len(inDefinition) == 4 for inDefinition in parameterDefinitions):
        raise ValueError("Input list elements must have the form: ['parameter', startValue, stopValue, stepValue]")
    
    # Unpack the transposed lists
    parameters, startValues, stopValues, numSteps = zip(*parameterDefinitions)
    # Convert the resulting tuples to lists
    parameters = list(parameters)
    startValues = list(startValues)
    stopValues = list(stopValues)
    numSteps = list(numSteps)

    #run the scan
    runNos = scanParam(parameters, startValues, stopValues, numSteps)
        
    return runNos

In [ ]:
def transparencyScan(baseParam, scanParam, changeGeo=True):
    """
    Performs a 1-dimensional scan of the simulation,
    looking for the field transparency -limiting case.

    Sets all parameters based on baseParam, 
    then linearly scans through a specified parameter.
    Simulations terminate if the transparency falls below the limit.

    Args:
        baseParam (dict): Dictionary containing
                          the base simulation parameters.
        scanParam (list): List of the form: 
                          ['parameterToScan', startValue, stopValue, numSteps]
                            - parameterToScan (str): The name of the parameter to vary.
                            - startValue (float): The starting value for the scan.
                            - stopValue (float): The ending value for the scan.
                            - numSteps (int): The number of steps in the scan.
        changeGeo (bool): Optional parameter to bypass geometry meshing.
                          (Efficient if geometry does not change between runs.)

    Returns:
        list: List of run numbers where the field transparency is greater than the limit.
              Empty if not all parameters are provided in baseParam or 
              if the scanning parameter is invalid
    """
    #Reset parameters before scan
    allParam = resetParam()

    for inParam in allParam:
        if inParam not in baseParam:
            print(f"Base parameter '{inParam}' not included.")
            return []

    if scanParam[0] not in allParam.keys():
        print('Invalid scanning parameter.')
        return []

    #Determine values for scan
    scanValues = np.linspace(scanParam[1], scanParam[2], scanParam[3])
    
    runNos = []
    print(f'Beginning transparency scan for: {scanParam[0]}')
    print(f'\tTransparency limit is: {baseParam['transparencyLimit']}')
    for inValue in scanValues:
        baseParam[scanParam[0]] = inValue

        #Ensure geometry is generated on at least the first sim
        if inValue == scanValues[0]:
            runNo = runSimulation(baseParam)
        else:
            runNo = runSimulation(baseParam, changeGeo)

        if runNo == -1:
            print(f'An error occured at: {scanParam[0]} = {inValue}')
            break

        #Terminate scan if transparency is below limit
        #TODO - this is depreciated!!!!
        if not checkFieldTransparency(runNo):
            print(f'Field transparency too low at: {scanParam[0]} = {inValue}. Ending scan')
            break

        runNos.append(runNo)

    #Done running - reset to defaults
    print(f'Done scan through {scanParam[0]}. Final run number: {getRunNumber()-1}')
    _ = resetParam()
    
    return runNos

In [ ]:
# Default run of a simulation.
## Determines a start-to-finish execution time.
## Individual program times are from log files.
if __name__ == '__main__':
    defaultSim = FIMS_Simulation()
    startTime = time.monotonic()
    defaultSim.runSimulation()
    endTime = time.monotonic()
    print(f'Test run takes ~{endTime-startTime:.1f}s.')

    '''
    Last time Tanner ran this test:
    Total time: ~888 s
        Gmsh: ~64 s
        ElmerGrid: ~1 s   
        ElmerSolver: ~700 s
        ElmerWeighting: ~34 s
        Garfield: ~90 s
    '''
    '''
    Last time James ran this test:
    Total time: ~2,827 s
        Gmsh: ~204 s
        ElmerGrid: ~1 s  
        ElmerSolver: ~878 s
        ElmerWeighting: ~50 s
        Garfield: ~1,693 s
    '''

In [ ]:
#if __name__ == '__main__':
#    scanValues([
#        ['fieldRatio', 30, 60, 4],
#        ['holeRadius', 16, 24, 3],
#        ['gridStandoff', 90, 110, 3]
#    ])
'''
[5365, 5366, 5367, 5368, 5369, 5370, 5371, 5372, 
 5373, 5374, 5375, 5376, 5377, 5378, 5379, 5380,
 5381, 5382, 5383, 5384, 5385, 5386, 5387, 5388,
 5389, 5390, 5391, 5392, 5393, 5394, 5395, 5396,
 5397, 5398, 5399, 5400]
'''

In [ ]:
def getOptimizerParam():
    """
    """
    params = [
        ['padLength', 40, 90],
        ['pitch', 150, 300],
        ['gridStandoff', 50, 150], 
        ['holeRadius', 50, 150],
    ]

    return params

In [ ]:
def runForOptimizer(simFIMS):
    """
    Runs the simulation, preserving the parameters.
    
    NOTE: This makes a copy of the parameters before running the
    simulation, as the simulation will reset them at the end.
    
    Args:
        simFIMS (FIMS_Simulation): An instance of the FIMS simulation class.
        
    Returns:
        int: The run number of the simulation that was executed.
    """
    saveParam = simFIMS.param.copy()
    runNumber = simFIMS.runSimulation()
    #Put saved parameters back
    simFIMS.param = saveParam

    return runNumber
    

In [ ]:
def calcIBF(runNumber):
    """
    Caluculates the ion backflow fraction for a given simulation.
    """

    simData = runData(runNumber)

    '''
    Do things to calculate IBF from simulation data here
    '''

    return ionBackflowFraction

In [ ]:
def getIBF(simFIMS):
    """
    Orchestrates the process of running a simulation and calculating
    the Ion Backflow Fraction (IBF) from the results.
    
    Args:
        simFIMS (FIMS_Simulation): An instance of the FIMS simulation class
                                   with its parameters set for a specific run.

    Returns:
        float: The calculated Ion Backflow Fraction.
    """
    # Get the minimum field ratio for the current parameters.
    minFieldRatio = getMinFieldRatio(simFIMS.param) ##James is working on this
    simFIMS.param['fieldRatio'] = minFieldRatio

    #Run the simulation
    runNumber = runForOptimizer(simFIMS)

    #Calculate the IBF
    IBF = calcIBF(runNumber)

    return IBF

In [ ]:
def objectiveForIBF(optimizerParam, simFIMS, inputList):
    """
    Objective function to optimize for minimum IBF.

    Updates the simulation's parameter dictionary using the values in the optimizer
    parameter array. Then gets the current IBF, prints it to monitor convergence,
    and returns it for the optimizer to minimize.
    
    Args:
        optimizerParam (np.array): The flat array of parameters from the optimizer.
        simFIMS (FIMS_Simulation): An instance of the FIMS simulation.
        inputList (list): A list of parameter names, matching the order of x.
    
    Returns:
        float: The IBF value to be minimized.
    """
    # Unpack the optimizer array into the simulation's parameter dictionary.
    for i, inParam in enumerate(inputList):
        simFIMS.param[inParam] = optimizerParam[i]
        
    # Get the Ion Backflow Fraction
    resultIBF = getIBF(simFIMS)
    #Print to monitor convergence
    print(f'Current IBF: {resultIBF:.4f}')
        
    return resultIBF

In [ ]:
def optimizeForIBF():
    """
    Runs an optimization routine to find the FIMS parameters that minimize 
    the Ion Backflow Fraction (IBF).

    Returns:
        dict: A dictionary containing:
            - params: Dictionary of optimal FIMS parameters.
            - IBFValue: Final minimum IBF value.
            - success: Boolean representing the success status of minimization.
    """
    simFIMS = FIMS_Simulation()

    #If setting a constant parameter different than default, do so here.
    simFIMS.param['numFieldLine'] = 0 #Field lines unecessesary for optimizer
    

    #Get optimizer parameters and bounds
    params = getOptimizerParam()
    inputList = [p[0] for p in params]
    minBounds = [p[1] for p in params]
    maxBounds = [p[2] for p in params]

    #Set bounds for variables
    optimizerBounds = Bounds(minBounds, maxBounds)

    #Set initial guess as default values
    optimizerParams = [simFIMS.param[inParam] for inParam in inputList]
    initialGuess = np.array(optimizerParams)

    print('Beginning optimization...')
    result = minimize(
        fun=objectiveForIBF,
        x0=initialGuess,
        args=(simFIMS, inputList),
        method='L-BFGS-B',
        bounds=optimizerBounds
    )
    print('----- Optimization complete. -----')

    #Put results into simulation instance
    for i, inParam in enumerate(inputList):
        simFIMS.param[inParam] = result.x[i] 

    print(f'Optimal FIMS parameters for IBF = {result.fun}:')
    print(simFIMS)

    resultVals = {
        'params': simFIMS.param, 
        'ibf_value': result.fun, 
        'success': result.success
    }

    return resultVals